# Note ROM Frequency deviations

To efficiantly calculate the note frequencies, a ROM is used to store the half counterperiods of a single octave. This notebook shows which frequencies are best to be stored in the ROM to minimize the frequency deviation across the full MIDI note range (21-127).

## Choosing the Clock Frequency

Two modules heavily depend on the clock frequency timing:

- `rx` Uart Reciever: the standart midi baud rate of 31.25kBd must be achievable reliably.
- `note2cnt` Note to Counter Period Converter: the clock frequency determines the maximum achievable frequency resolution.

The oscillator uses a counter to generate the output frequency. The Lowest Frequency requires the highest counter period. To find a compromise between frequency accuracy (higher clock frequency, therefor larger counter registers) and space efficiency, the frequency must be chosen, such that the highest counter period fits into a $2^n$ bit register.

To further save space, only half the counterperiod has to be stored in ROM, as the oscillator toggles the output pin every half period (output wave has 50% duty cycle). Therefor, the frequnecy can be doubled further without sacrificing register size.

The optimal clock frequency can be calculated as follows:

$$
f_{\mathrm{clk}} = 2\cdot f_{\mathrm{osc, min}} \cdot 2^{BW} = f_{\mathrm{osc, min}} \cdot 2^{BW+1}
$$

For 

- $BW = 16$ and
- $f_{\mathrm{osc, min}} = 27.5\,\mathrm{Hz}$ (lowest midi note frequency),
- the formula yields $f_{\mathrm{clk}} = 3604480\,\mathrm{Hz}$. 

To prevend counter overflows, the clock frequency is chosen slightly lower to $f_{\mathrm{clk}} = 3500000\,\mathrm{Hz}$. The lowest note is then represented by $64814.(81481)$. The trucation error is very small, with:

$$
f_{\mathrm{osc, min, actual}} = \frac{f_{\mathrm{clk}}}{2 \cdot 64814} \approx 27.000339\,\mathrm{Hz} \implies \Delta f \approx -3.39\cdot 10^{-4}\,\mathrm{Hz}
$$

In [88]:
f_clk_hz = 3500000 # <- Optimal to achive close to 16 bit counter for 20Hz minimum frequency
f_clk_half_hz = f_clk_hz / 2
cnt_bw = 16

The [Piano Key Frequencies](https://en.wikipedia.org/wiki/Piano_key_frequencies) over the MIDI-Note range are calculated as follows: 

In [229]:
midi_note_max = 128
midi_note_min = 21
midi_note_a4 = 69
keys_per_octave = 12
f_a4_hz = 440

full_frequency_range = [f_a4_hz * (2 ** ((n - midi_note_a4) / keys_per_octave)) for n in range(midi_note_min, midi_note_max+1)]

n_octaves = int((midi_note_max - midi_note_min + 1) // keys_per_octave + 1)

In [215]:
def get_octave_freqs(index):
	return full_frequency_range[index * 12 : (index + 1) * keys_per_octave]

def octave_to_cnts(octave_freqs):
	return [int(f_clk_half_hz // f) for f in octave_freqs]

## Choosing which Octaves to store

To get the other frequencies, the stored octaves are either divided or multiplied by two (left or right shift), depending on the octave. This leads to frequency deviations, as the counter periods are integer values and truncation errors occur.


In [216]:
import numpy as np

def octave_diff(src_octave, dest_octave):
	octave_freqs = get_octave_freqs(src_octave)
	octave_cnts = octave_to_cnts(octave_freqs)
	shift_amount = abs(dest_octave - src_octave)

	if src_octave < dest_octave:
		# shifting up
		shifted_cnts = np.array(octave_cnts) >> shift_amount
	else:
		# shifting down
		shifted_cnts = np.array(octave_cnts) << shift_amount

	synth_freqs = f_clk_hz / shifted_cnts / 2;
	octave_goal_freqs = np.array(get_octave_freqs(dest_octave))
	deviation = synth_freqs - octave_goal_freqs
	
	print(f"Octave shift from {src_octave} to {dest_octave}:")
	print("| MIDI | CNT | Goal Freq | Synth Freq | Deviation |")
	for i in range(keys_per_octave):
		print(f"| {i + dest_octave * 12 + midi_note_min} | {shifted_cnts[i]} | {octave_goal_freqs[i]:4.4f} | {synth_freqs[i]:4.4f} | {deviation[i]:2.4f} |")

	rom_sizes = np.ceil(np.log2(shifted_cnts))
	max_rom_size = int(np.max(rom_sizes))
	entries_with_max_size = np.sum(rom_sizes == max_rom_size)
	storage_efficiency = (entries_with_max_size / keys_per_octave) * 100
	
	print("\nROM Size if these counter values were stored:")
	print(f"ROM size needed: {max_rom_size} bits")
	print(f"{entries_with_max_size} Entries require this size")
	print(f"{storage_efficiency}% Storage efficiency")

The higher the octave, the lower the cost of ROM storage due to smaller counter periods. However low counter periods have a larger error which accumulates when shifting to lower octaves.

The Base error can be calculated by comparing the same octaves. Then an octave with an acceptable error can be chosen to be stored in ROM.

In [231]:
octave_diff(8, 0)

Octave shift from 8 to 0:
| MIDI | CNT | Goal Freq | Synth Freq | Deviation |
| 21 | 63488 | 27.5000 | 27.5643 | 0.0643 |
| 22 | 59904 | 29.1352 | 29.2134 | 0.0782 |
| 23 | 56576 | 30.8677 | 30.9318 | 0.0641 |
| 24 | 53504 | 32.7032 | 32.7078 | 0.0046 |
| 25 | 50432 | 34.6478 | 34.7002 | 0.0524 |
| 26 | 47616 | 36.7081 | 36.7524 | 0.0443 |
| 27 | 44800 | 38.8909 | 39.0625 | 0.1716 |
| 28 | 42240 | 41.2034 | 41.4299 | 0.2265 |
| 29 | 39936 | 43.6535 | 43.8201 | 0.1666 |
| 30 | 37632 | 46.2493 | 46.5030 | 0.2537 |
| 31 | 35584 | 48.9994 | 49.1794 | 0.1800 |
| 32 | 33536 | 51.9131 | 52.1827 | 0.2696 |

ROM Size if these counter values were stored:
ROM size needed: 16 bits
12 Entries require this size
100.0% Storage efficiency


### Testing if the Frequnency Offset is acceptable

The Offset is determied by ear. A Synthesizer (frequency Generator) is used to generate the calculated expected output frequency. This test showed that all errors are acceptable. As the larger errors only occur at higher frquencies, the ear is less sensitive to these deviations. The error from octave 8 is the largest and starts to become audible, with the benefit of requireing only 8 bit storage per note.

### ROM Size and Storage Efficiency

With a maximum counter value close to a power of two, the ROM efficiency can be maximized, as each octave is a factor of two apart.

In [232]:
middle_octave_cnts = octave_to_cnts(get_octave_freqs(8))
for i in range(keys_per_octave):
	print(f"noteRom[{i}] = {middle_octave_cnts[i]};")

noteRom[0] = 248;
noteRom[1] = 234;
noteRom[2] = 221;
noteRom[3] = 209;
noteRom[4] = 197;
noteRom[5] = 186;
noteRom[6] = 175;
noteRom[7] = 165;
noteRom[8] = 156;
noteRom[9] = 147;
noteRom[10] = 139;
noteRom[11] = 131;
